In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pymorphy2
import pyprind
import nltk
import json
import os
import re

In [2]:
# загрузка датасета
df = pd.read_csv("data/train.csv", encoding="utf-8", index_col="index")
df.shape

(30000, 3)

In [3]:
train = df.query("target != -1")
print(f"{train.shape=}")
display(train.head())

train.shape=(15650, 3)


,name,description,target
index,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212
179260831,кассир,Правила приема и проведения расчетных и конвер...,5223
39608878,оператор машинного доения,"Выполнять все виды работ, связанные с производ...",6121


## Предобработка 1
Вам будут предоставлены 30 тыс. вакансий для обучения модели - но только для 15 тыс. из них снабжены метками класса. Как использовать оставшиеся 15 тыс. примеров - решает каждая команда индивидуально. Для демонстрации просто выбросим их.

### Дополняем описаниями требований вакансий
Для вашего удобства произведена первичная преобработка текстового описания, из которого были выделены логические блоки: "обязанности", "требования" и т.д. Предобработанные описания хранятся в формате `JSON`. Загрузим один из файлов и посмотрим, как выглядят эти данные.

In [4]:
# открываем json с описаниями

path_list = ["data/vacancy_descriptions/1_parsed.json",
             "data/vacancy_descriptions/2_parsed.json",
             "data/vacancy_descriptions/3_parsed.json", 
             "data/vacancy_descriptions/4_parsed.json",
             "data/vacancy_descriptions/5_parsed.json"]

def json_loader(path):
    
    with open(
        path, "r", encoding="utf8"
    ) as fp:
        descriptions = json.load(fp)
    
    return descriptions
    

1. Для каждой вакансии возьмём описания: Position, Обязаности, Требования

`description["ID"]: r[:1]` - r[:1] количество строк описания из обязаностей которые мы добавим в датасет

In [5]:
def respons_builder(path_list):
    responsibilities_ans = pd.Series()
    requirements_ans = pd.Series()
    positions_ans = pd.Series()
    for path in path_list:
        print(path)

        descriptions = json_loader(path)
        responsibilities = pd.Series({
            description["ID"]: " ".join(r[:]) 
            if (r := description["Content"].get("Обязанности")) is not None
            else None
            for description in descriptions
        }, name="responsibilities")
        
        requirements = pd.Series({
            description["ID"]: " ".join(r[:]) 
            if (r := description["Content"].get("Требования")) is not None
            else None
            for description in descriptions
        }, name="requirements")
        
        positions = pd.Series({
            description["ID"]: r[:]
            if (r := description["Position"]) is not None
            else None
            for description in descriptions
        }, name="positions")
            
        responsibilities_ans = pd.concat([responsibilities_ans, responsibilities], axis=0)
        requirements_ans = pd.concat([requirements_ans, requirements], axis=0)
        positions_ans = pd.concat([positions_ans, positions], axis=0)
        
    return responsibilities_ans, requirements_ans, positions_ans
    

responsibilities_ans, requirements_ans, positions_ans = respons_builder(path_list)

display(responsibilities_ans.head(10)), responsibilities_ans.shape, responsibilities_ans.isnull().sum()

<ipython-input-5-27892ad9f24c>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  responsibilities_ans = pd.Series()
<ipython-input-5-27892ad9f24c>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  requirements_ans = pd.Series()
<ipython-input-5-27892ad9f24c>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  positions_ans = pd.Series()


data/vacancy_descriptions/1_parsed.json
data/vacancy_descriptions/2_parsed.json
data/vacancy_descriptions/3_parsed.json
data/vacancy_descriptions/4_parsed.json
data/vacancy_descriptions/5_parsed.json


363107946                                                 None
363125198                                                 None
363144355                                                 None
363340232                                                 None
363383054    Организовывать и проводить просмотры/показы об...
363982236                                                 None
364010638                                                 None
364022326                                                 None
364029440                                                 None
367089693                                                 None
dtype: object

(None, (47566,), 5556)

2. Добавим еще столбецы с описанием в таблицу с обучающей выборкой.

In [6]:
train["responsibilities"] = responsibilities_ans
train["requirements"] = requirements_ans
train["positions"] = positions_ans
display(train.head(3))
display(train.info())

<ipython-input-6-715ebf1b2f20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["responsibilities"] = responsibilities_ans
<ipython-input-6-715ebf1b2f20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["requirements"] = requirements_ans
<ipython-input-6-715ebf1b2f20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,name,description,target,responsibilities,requirements,positions
index,,,,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223,":принимать и выдавать заказы, расчет с гостями...",от ТЕБЯ:желание обучиться работе с оборудован...,"Кассир в пиццерию, г. Витебск"
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223,:Рассказывать людям о Yota Продавать устройств...,None,Продавец-консультант Yota (ТЦ Галерея)
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212,ручная дуговая сварка (рд) сварка и монтаж мет...,оформление согласно тк рф стабильная выплата з...,"Электросварщик НАКС СК (п.1, г. Норильск)"


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15650 entries, 169939030 to 39324585
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              15650 non-null  object
 1   description       15645 non-null  object
 2   target            15650 non-null  int64 
 3   responsibilities  13339 non-null  object
 4   requirements      12643 non-null  object
 5   positions         14936 non-null  object
dtypes: int64(1), object(5)
memory usage: 855.9+ KB


None

3. Объединим два столбца: `name` и `responsibilities`,`requirements`, `position`. Строки в столбцах которых  содержатся пропуски, заполним при помощи метода `fillna`. Приведем полученный столбец к нижнему регистру.

In [7]:
train["full_description"] = (
    train["name"] + " " + train["requirements"].fillna("") + " " + train["responsibilities"].fillna("")
    + " " + train["positions"].fillna("") 
).map(str.lower)

display(train.head(10))

<ipython-input-7-fe09f83171bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["full_description"] = (


,name,description,target,responsibilities,requirements,positions,full_description
index,,,,,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223,":принимать и выдавать заказы, расчет с гостями...",от ТЕБЯ:желание обучиться работе с оборудован...,"Кассир в пиццерию, г. Витебск",кассир в пиццерию г витебск от тебя:желание о...
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223,:Рассказывать людям о Yota Продавать устройств...,None,Продавец-консультант Yota (ТЦ Галерея),продавец консультант yota (тц галерея) :расск...
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212,ручная дуговая сварка (рд) сварка и монтаж мет...,оформление согласно тк рф стабильная выплата з...,"Электросварщик НАКС СК (п.1, г. Норильск)",электросварщик накс ск (п 1 г норильск) оформл...
179260831,кассир,Правила приема и проведения расчетных и конвер...,5223,Правила приема и проведения расчетных и конвер...,"Знание програграммы 1 С, добросовестное отноше...",Кассир,"кассир знание програграммы 1 с, добросовестное..."
39608878,оператор машинного доения,"Выполнять все виды работ, связанные с производ...",6121,"Выполнять Все Виды Работ, Связанные С Производ...",Ответственность,Оператор машинного доения,оператор машинного доения ответственность выпо...
278741128,повар шашлычник,<ul><li>Приготовление шашлыка</li><li>Поддержа...,5120,приготовление шашлыкаподдержание чистоты на ра...,опыт работы в аналогичной должности от 3-х лет...,Повар-шашлычник,повар шашлычник опыт работы в аналогичной долж...
66104737,врач кардиолог,"выполнение работ по оказанию постоянной, неотл...",2211,"выполнение работ по оказанию постоянной, неотл...","высшее медицинское образование по профилю, нал...",Врач-кардиолог,врач кардиолог высшее медицинское образование ...
32686340,доярка дояр,<strong>Обязанности:</strong> <ul> <li>Дойка и...,6121,Дойка и уход за стадом 50 двойных коз Дойка ап...,Аккуратность Знание и умения Любовь к животным...,Доярка/дояр,доярка дояр аккуратность знание и умения любов...
27380570,массажист,<strong>Обязанности:</strong> <ul> <li>Выполне...,3255,Выполнение классического лечебного массажа Точ...,Медицинское образование Действующий сертификат...,Массажист,массажист медицинское образование действующий ...


## Предобработка: 2. Очистка текстовых данных

Текстовые данные часто содержат много лишней информации, которая мешает модели обучаться. Почистим данные: для демонстрации уберем знаки препинания из текста. Для этого воспользуемся механизмом регулярных выражений.

Регулярные выражения (regular expressions, regexp) — формальный язык, используемый в программах, работающих с текстом, для поиска и осуществления манипуляций с подстроками в тексте, основанный на использовании шаблонов, содержащих метасимволы (символы-джокеры, англ. wildcard characters).

Очистка текстовых данных, удаление всех нежелательных символов. 
Удаление всех небуквеных символов. Текстовые данные часто содержат много лишней информации, которая мешает модели обучаться. Почистим данные: для демонстрации уберем знаки препинания из текста. Для этого воспользуемся механизмом регулярных выражений.

`<[^>]*>` - удаляем всю HTML размметку  из текста. 

`[\W]+` - приводим все буквы к нижнему регистру 

In [8]:
#  функция очистки
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', ''))
    return text

In [9]:
# применим очистку данных ко всем данным в столбце "review"
train["full_description"] = train["full_description"].apply(preprocessor)

<ipython-input-9-4f01309e7e60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["full_description"] = train["full_description"].apply(preprocessor)


In [ ]:
display(train["full_description"].head(2))

## 3. Предобработка документов в лексемы


###  3.1 Разбиваем текст на токены  + добавляем упрощённые формы 

In [12]:
# пример разбиваем текст на токены  + добавляем упрощённые формы 

morph = pymorphy2.MorphAnalyzer()

# пример разделение предложение на слова
def tokenizer(text):
    return text.split()

# пример сокращает слова до корневой формы
def tokenizer_morphy(text):
    return [morph.parse(word)[0].normal_form for word in text.split()]


In [ ]:
# проверка работы токенезатора и упрошения формы слова
tokenizer_morphy('кассир в пиццерию г витебск принимать')


###  3.2 Удаление стоп-слов

Стоп-слова, такие слова которые не несут в себе полезной информации

In [13]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\_username_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# исключение стоп слова для РУСССКИЕ СТОП СЛОВА
stop = stopwords.words('russian')
[w for w in tokenizer("кассир в пиццерию г витебск принимать") if w not in stop]

['кассир', 'пиццерию', 'г', 'витебск', 'принимать']

## 4. Построение модели

In [15]:
X_train, y_train = train["full_description"], train["target"]
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.80, random_state=42)

In [16]:
X_train, y_train.head()

(index
 208567164    заведующий фельдшерско акушерским пунктом фель...
 168194654    врач терапевт участковый наличие сертификата с...
 30933029     продавец кассир иваново улица дзержинского 12 ...
 165512246    бухгалтер опыт работы более 5 лет 1с бухгалтер...
 171374966    логист знание программы 1с опыт работы с транс...
                                    ...                        
 170920388    дворник ответственный дисциплинированный уборк...
 255988612    заместитель директора магазина тавда ленина 46...
 111422926    фрезеровщик опыт работы от трех лет чтение чер...
 126596983    оператор по искусственному осеменению животных...
 114907480    бухгалтер знание осн усн уверенный пользовател...
 Name: full_description, Length: 12520, dtype: object,
 index
 208567164    2240
 168194654    2211
 30933029     5223
 165512246    2411
 171374966    4323
 Name: target, dtype: int64)

* Воспользуемся Tfidf
Подход для снижения веса часто встречающихся слов в векторах признаков, обратная частота документа (term frequency - inverse document freq - tf-idf).

* устанавим n_jobs=-1 (вместо n_jobs=1), чтобы задействовать все свободные процессорные ядра и ускорить решетчатый поиск



In [20]:
# !!!тюнинг параметров
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

small_param_grid = [{'vect__ngram_range': [(1, 2)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer_morphy, tokenizer],
                     'vect__use_idf':[True, False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 0.1],
                'clf__max_iter':[100, 150]
                    }
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(multi_class='multinomial', solver='lbfgs'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=3,
                           verbose=1,
                           n_jobs=-1)

In [15]:
# с оптимизатором newton_cg (вторая модель)

# tfidf = TfidfVectorizer(strip_accents=None,
#                         lowercase=False,
#                         preprocessor=None)

# small_param_grid = [{'vect__ngram_range': [(1, 3)],
#                      'vect__stop_words': [stop],
#                      'vect__tokenizer': [tokenizer_morphy],
#                      'vect__use_idf':[True],
#                      'vect__norm':[None],
#                      'clf__penalty': ['l2'],
#                      'clf__C': [0.1],
#                      'clf__max_iter':[20, 50, 100]
                     
#                     }

#               ]

# lr_tfidf = Pipeline([('vect', tfidf),
#                      ('clf', LogisticRegression(multi_class='multinomial', solver ='newton-cg'))])

# gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
#                            scoring='accuracy',
#                            cv=3,
#                            verbose=1,
#                            n_jobs=-1)

In [21]:
# ОСТОРОЖНО !!! ДОЛГИЕ ВЫЧИСЛЕНИЯ
gs_lr_tfidf.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


c:\users\_username_\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
c:\users\_username_\appdata\local\programs\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
c:\users\_username_\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(multi_class='multinomial'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 2)], 'vect__norm': [None],
                          'vect__stop_words': [['и', 'в', 'во', 'не', 'что',
                                                'он', 'на', 'я', 'с', 'со',
                                                'как', 'а', 'то', 'все', 'она',
                                                'так', 'его', 'но', 'да', 'ты',
                                                'к', 'у', 'же', 'вы', 'за',
                                                'бы', 'по', 'только', 'ее',
                                                'мне', ...]],
                        

In [22]:
# TRAIN ACCURACY
# при бинарной класс 0.965
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 2), 'vect__norm': None, 'vect__stop_words': ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'пос

In [23]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_validation, y_validation):.3f}')
# print(f'Train Accuracy: {clf.score(X_train, y_train):.3f}')

Test Accuracy: 0.961


In [24]:
# сохранение модели
import joblib
# joblib.dump(gs_lr_tfidf.best_estimator_, 'lbfgs_model_lr_last.pkl')

# загрузка модели 
# clf = joblib.load("newton-cg_model_lr.pkl")

['lbfgs_model_lr_last.pkl']

__РЕЗЮМЕ__


logress_lbfgs Acc 0.993 --> Score 0.53 

logress_lbfgs на расширенных данных Acc 0.996--> Score 0.54 

logress_lbfgs + respons+ requarament Acc 0.996 --> !!!Score 0.54291

logress_lbfgs + respons+ requarament + position  Acc -->0.998 !!! Score: 0.55223

logress_lbfgs + respons+ requarament + position + расширенные данные Acc 0.998--> !!! 0.5485 ??

logress_lbfgs(тюнинг) + respons+ requarament + position + logress_newton_cg(тюнинг): CV Accuracy: 0.993 -->0.569


## 5. Расширение датасета + дообучение + новое предсказание на расширенном датасете


In [32]:
big_train  = pd.read_csv("data/train.csv", encoding="utf-8", index_col="index")
big_train = big_train.query("target ==-1")
big_train["responsibilities"] = responsibilities_ans
big_train["requirements"] = requirements_ans
big_train["positions"] = positions_ans
big_train["full_description"] = (
    big_train["name"] + " " + big_train["requirements"].fillna("") + " " + big_train["responsibilities"].fillna("")
    + " " + big_train["positions"].fillna("") 
).map(str.lower)

big_train["full_description"] = big_train["full_description"].apply(preprocessor)
X_big_train = big_train["full_description"]


In [33]:
display(X_big_train.head())
X_big_train.shape

index
324865089    продавец кассир з п 2 раза в месяц своевременн...
169467135    продавец мила шевченко 17 коммуникабельность а...
31956044     начальник строительного участка знание и соблю...
36781653      продавец кассир санкт петербург бухарестская 31 
49435567     главный бухгалтер полное ведение бухгалтерског...
Name: full_description, dtype: object

(14350,)

In [34]:
big_train.to_csv("data/big_data_train.csv")

In [35]:
# clf = gs_lr_tfidf.best_estimator_
big_y_pred = clf.predict(X_big_train)
big_y_pred.shape

(14350,)

In [36]:
big_train.reset_index(inplace= True )
submission = big_train[["index"]].assign(target=big_y_pred)
display(submission.head(4))
submission.to_csv("data/big_train.csv")

,index,target
0,324865089,5223
1,169467135,5223
2,31956044,1323
3,36781653,5223


In [37]:
# создание нового трейна1
new_df1 = pd.read_csv("data/train.csv", encoding="utf-8", index_col="index")
new_df1 = new_df1.query("target !=-1")
new_df1.to_csv("data/new_train1.csv")

In [38]:
# создание нового трейна 2
new_df2 = pd.read_csv("data/train.csv", encoding="utf-8", index_col="index")
new_df2 = new_df2.query("target ==-1")
big_df = pd.read_csv("data/big_train.csv", encoding="utf-8", index_col="index")
new_df2["target"] = big_df["target"]
new_df2.to_csv("data/new_train2.csv")

In [39]:
df1 = pd.read_csv("data/new_train1.csv", encoding="utf-8", index_col="index")
df2 = pd.read_csv("data/new_train2.csv", encoding="utf-8", index_col="index")
new_df = pd.concat([df1, df2], axis=0)

In [40]:
new_df.to_csv("data/absolute_train.csv")

###  5.1  Абсолютный датасет

In [41]:
absolute_df = pd.read_csv("data/absolute_train.csv", encoding="utf-8", index_col="index")
display(absolute_df.head())

,name,description,target
index,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212
179260831,кассир,Правила приема и проведения расчетных и конвер...,5223
39608878,оператор машинного доения,"Выполнять все виды работ, связанные с производ...",6121


### 5.2 дообучение и предсказание на расширенных данных


In [42]:
absolute_df  = pd.read_csv("data/absolute_train.csv", encoding="utf-8", index_col="index")
absolute_df["responsibilities"] = responsibilities_ans
absolute_df["requirements"] = requirements_ans
absolute_df["positions"] = positions_ans
absolute_df["full_description"] = (
    absolute_df["name"] + " " + absolute_df["requirements"].fillna("") + " " + absolute_df["responsibilities"].fillna("")
    + " " + absolute_df["positions"].fillna("") 
).map(str.lower)

absolute_df["full_description"] = absolute_df["full_description"].apply(preprocessor)
X_absolute_train, y_absolute_train = absolute_df["full_description"], absolute_df["target"]

In [43]:
absolute_df.to_csv("data/absolute_full_data.csv")

In [44]:
absolute_df  = pd.read_csv("data/absolute_full_data.csv", encoding="utf-8", index_col="index")
X_absolute_train, y_absolute_train = absolute_df["full_description"], absolute_df["target"]

In [45]:
X_abs_train, X_abs_validation, y_abs_train, y_abs_validation = train_test_split(X_absolute_train, y_absolute_train, train_size=0.85, random_state=42)

In [46]:
display(absolute_df.head(1)), absolute_df.shape

,name,description,target,responsibilities,requirements,positions,full_description
index,,,,,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223,":принимать и выдавать заказы, расчет с гостями...",от ТЕБЯ:желание обучиться работе с оборудован...,"Кассир в пиццерию, г. Витебск",кассир в пиццерию г витебск от тебя желание об...


(None, (30000, 7))

In [ ]:
# разбивка датасета
# X_xgtrain, y_xgtrain = xgtrain["full_description"], xgtrain["target"]
# X_train, X_validation, y_train, y_validation = train_test_split(X_xgtrain, y_xgtrain, train_size=0.85, random_state=42)

In [51]:
# переобучение для newton-cg_model_lr.pkl для предсказания на lbfgs_model_lr_last.pkl
# clf = gs_lr_tfidf.best_estimator_
clf = joblib.load("newton-cg_model_lr.pkl")
clf.fit(X_abs_train, y_abs_train)


c:\users\_username_\appdata\local\programs\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
c:\users\_username_\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False, ngram_range=(1, 3), norm=None,
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',
                                             'но', 'да', 'ты', 'к', 'у', 'же',
                                             'вы', 'за', 'бы', 'по', 'только',
                                             'ее', 'мне', ...],
                                 tokenizer=<function tokenizer_morphy at 0x000002E95B5F8310>)),
                ('clf',
                 LogisticRegression(C=0.1, max_iter=20,
                                    multi_class='multinomial',
                                    solver='newton-cg'))])

In [52]:
print(f'CV Accuracy: {clf.score(X_absolute_train, y_absolute_train):.3f}')
print(f'CV Accuracy: {clf.score(X_abs_validation, y_abs_validation):.3f}')

CV Accuracy: 0.993
CV Accuracy: 0.972


In [53]:
# сохранение модели
# joblib.dump(clf, 'lbfgs+_newton_cg_model_lr.pkl')

['lbfgs+_newton_cg_model_lr.pkl']

## 6 Предсказание для KAGGLE

In [54]:
test = pd.read_csv("data/test.csv", encoding="utf-8", index_col="index")
display(test.head(5))
test.shape

,name,description
index,,
26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...
26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...
26467473,"продавец кассир (тц ""седанка сити"")",<p><strong>Твои обязанности:</strong></p> <ul>...
26468989,специалист по кадрам,"<p>Обязанности:</p> <p>- оформление приема, пе..."
26471705,администратор на телефоне,<strong>Обязанности:</strong> <ul> <li> <p>Обр...


(1090, 2)

In [55]:
test["responsibilities"] = responsibilities_ans
test["requirements"] = requirements_ans
test["positions"] = positions_ans
test["full_description"] = (
    test["name"] + " " + test["requirements"].fillna("") + " " + test["responsibilities"].fillna("")
    + " " + test["positions"].fillna("") 
).map(str.lower)

test["full_description"] = test["full_description"].apply(preprocessor)
X_test= test["full_description"]

In [56]:
display(test.head(2))

,name,description,responsibilities,requirements,positions,full_description
index,,,,,,
26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...,Выполнение поступающих заявок через приложение...,"Отличное знание Москвы и ПДД Знание ПК, умение...",Персональный водитель сервиса Wheely,персональный водитель сервиса wheely отличное ...
26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...,Поиск поставщиков Ведение перевозок от начало ...,Опыт работы не менее 1 года Хорошие коммуникат...,Менеджер по автоперевозкам,менеджер по автоперевозкам опыт работы не мене...


In [57]:
X_test = test["full_description"]
display(X_test.head(2))
X_test.shape

index
26461447    персональный водитель сервиса wheely отличное ...
26464220    менеджер по автоперевозкам опыт работы не мене...
Name: full_description, dtype: object

(1090,)

In [58]:
# clf = gs_lr_tfidf.best_estimator_
y_pred = clf.predict(X_test)

In [59]:
y_pred
# joblib.dump(clf, 'full_newton_cg_model_lr.pkl')

array([8322, 4323, 5223, ..., 5223, 8343, 4323], dtype=int64)

Оформить ввиде датафрейма с двумя столбццами index и target.

In [60]:
test.reset_index(inplace= True )
submission = test[["index"]].assign(target=y_pred)
display(submission.head(4))
submission.to_csv("lbfgs_newton_cg.csv", index=False)

,index,target
0,26461447,8322
1,26464220,4323
2,26467473,5223
3,26468989,1349


__Резюме__

Score
0.569